In [1]:
import numpy as np
from keras.datasets import mnist
from keras.models import Model
from keras.layers import Dense, Input
import matplotlib.pyplot as plt
import keras

DATAPATH = "../data/"
encoding_dim = 20 # 压缩后的维度

Using TensorFlow backend.


In [2]:
country_feature = np.load(DATAPATH + "features/country_feature.npy")
genre_feature = np.load(DATAPATH + "features/genre_feature.npy")
audio_feature = np.load(DATAPATH + "features/audio_feature.npy")
video_feature = np.load(DATAPATH + "features/video_feature.npy")

# 图像特征归一化
max = video_feature.max()
min = video_feature.min()
video_feature = (video_feature-min)/(max-min)

In [3]:
country_input = Input(shape=(country_feature.shape[1],))
genre_input = Input(shape=(genre_feature.shape[1],))
audio_input = Input(shape=(audio_feature.shape[1],))
video_input = Input(shape=(video_feature.shape[1],))

In [4]:
input_layer = keras.layers.concatenate([country_input, genre_input, audio_input, video_input])

# encoder layers
encoded = Dense(128, activation='relu')(input_layer)
encoded = Dense(64, activation='relu')(encoded)
encoded = Dense(10, activation='relu')(encoded)
encoder_output = Dense(encoding_dim)(encoded)

# decoder layers
decoded = Dense(10, activation='relu')(encoder_output)
decoded = Dense(64, activation='relu')(decoded)
decoded = Dense(128, activation='relu')(decoded)
decoded = Dense(784, activation='relu')(decoded)

In [5]:
# output layers
'''
softmax [0,1]，每一行的和为1
sigmoid [0,1]，每一行的元素大小相互独立
tanh [-1,1]，每一行的元素大小互相独立
'''
country_output = Dense(country_feature.shape[1], activation='softmax')(decoded)
genre_output = Dense(genre_feature.shape[1], activation='sigmoid')(decoded)
audio_output = Dense(audio_feature.shape[1], activation='tanh')(decoded)
video_output = Dense(video_feature.shape[1], activation='sigmoid')(decoded)

In [6]:
# construct the autoencoder model
autoencoder = Model(input=[country_input, genre_input, audio_input, video_input],
                    output=[country_output, genre_output, audio_output, video_output])

# construct the encoder model for plotting
encoder = Model(input=[country_input, genre_input, audio_input, video_input], output=encoder_output)

# compile autoencoder
autoencoder.compile(optimizer='adam', loss='mse')

# training
autoencoder.fit([country_feature, genre_feature, audio_feature, video_feature],
                [country_feature, genre_feature, audio_feature, video_feature],
                epochs=100, batch_size=256, shuffle=True, verbose=1)

c:\users\wang\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:4: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=[<tf.Tenso..., outputs=[<tf.Tenso...)`
  after removing the cwd from sys.path.
c:\users\wang\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:7: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=[<tf.Tenso..., outputs=Tensor("de...)`
  import sys


Epoch 1/100
2/2 [==============================] - 1s 541ms/step - loss: 0.4720 - dense_9_loss: 0.0263 - dense_10_loss: 0.2501 - dense_11_loss: 0.0127 - dense_12_loss: 0.1830
Epoch 2/100
2/2 [==============================] - 0s 75ms/step - loss: 0.4708 - dense_9_loss: 0.0263 - dense_10_loss: 0.2492 - dense_11_loss: 0.0125 - dense_12_loss: 0.1828
Epoch 3/100
2/2 [==============================] - 0s 146ms/step - loss: 0.4692 - dense_9_loss: 0.0263 - dense_10_loss: 0.2479 - dense_11_loss: 0.0124 - dense_12_loss: 0.1826
Epoch 4/100
2/2 [==============================] - 0s 68ms/step - loss: 0.4669 - dense_9_loss: 0.0263 - dense_10_loss: 0.2460 - dense_11_loss: 0.0122 - dense_12_loss: 0.1824
Epoch 5/100
2/2 [==============================] - 0s 64ms/step - loss: 0.4633 - dense_9_loss: 0.0263 - dense_10_loss: 0.2430 - dense_11_loss: 0.0119 - dense_12_loss: 0.1821
Epoch 6/100
2/2 [==============================] - 0s 56ms/step - loss: 0.4585 - dense_9_loss: 0.0263 - dense_10_loss: 0.2390 - 

In [7]:
test = autoencoder.predict([country_feature, genre_feature, audio_feature, video_feature])
test

[array([[1.72446980e-05, 4.15435352e-05, 1.31135515e-04, 9.03740292e-05,
         3.81150749e-05, 1.64745477e-04, 2.05788936e-04, 5.56637569e-05,
         7.61421979e-05, 7.08383159e-05, 3.59833975e-05, 6.37029734e-05,
         7.34239802e-05, 1.00473597e-04, 5.72171230e-05, 6.52783565e-05,
         2.83385798e-05, 1.01175516e-04, 2.75479106e-05, 3.50991249e-05,
         1.24962908e-05, 1.50923333e-05, 4.59184630e-05, 8.11661957e-05,
         3.19339597e-05, 4.20388096e-05, 3.66497006e-05, 7.43155470e-05,
         6.04571942e-05, 4.35349284e-05, 8.22700968e-05, 2.10582602e-05,
         9.85454917e-01, 4.42112250e-05, 1.23785455e-02, 4.31843982e-05,
         5.23743038e-05],
        [1.56140891e-06, 5.24199640e-06, 4.25736289e-06, 7.93640356e-06,
         8.19492129e-07, 6.41141150e-06, 9.03252385e-06, 8.01886983e-07,
         6.37344965e-06, 3.37057895e-06, 8.23987250e-07, 1.29056082e-06,
         1.91299796e-06, 2.57783631e-06, 3.46539923e-06, 5.21271886e-06,
         1.51485676e-06, 

In [8]:
test_encoder = encoder.predict([country_feature, genre_feature, audio_feature, video_feature])
test_encoder


array([[-3.2792664 ,  2.0777001 , -2.4550774 ,  2.7520745 ,  1.3064508 ,
         1.7742621 ,  1.7763183 , -1.0036951 , -1.279995  ,  2.253398  ,
         0.77858496, -1.4887658 ,  1.212124  ,  5.802516  , -1.4556428 ,
         2.2429197 , -3.9404824 , -1.9668274 ,  1.964121  ,  4.5578527 ],
       [-4.738972  ,  4.089715  , -6.016193  ,  2.5938637 ,  3.7600367 ,
         3.591629  ,  2.3513103 , -1.4433155 , -2.1614428 ,  3.0630052 ,
        -2.3878677 , -2.8513517 ,  1.0529511 ,  9.858205  ,  0.02260466,
         1.7538059 , -7.7236524 , -1.7231075 ,  2.596217  ,  5.5566773 ]],
      dtype=float32)